# EBT Cohort Simulation for the Simple Wheat Model
<div style="margin-top: 20px; margin-left: 20px; font-size: 90%; color: #888888">
by Adriana M. Ortiz Aquino, Elizabeth J. Hale, Lauren M. White, Md. Atiqul Islam, Saikat Kuili  
    
Based on EBT Model by Nathan Albin &lt;albin@ksu.edu&gt;<br>
</div>

In homework 5 we created a model class and then another subclass for the simple wheat model using the equations 
$$
\begin{align}
\frac{d}{dt} TT &= \max (T_{avg}-T_{base},0) \\
\frac{d}{dt} LAI &= \begin{cases} &\alpha \max (T_{avg}-T_{base},0) LAI\cdot \max(LAI_{max}-LAI,0) \quad \text{ if } TT\leq TTL \\
&0 \quad \text{otherwise}
\end{cases} \\
\frac{d}{dt} B &= \begin{cases}  &RUE[1-e^{-K\cdot LAI}]I \quad \text{ if } TT\leq TTM \\
&0 \quad \text{otherwise}
\end{cases}
\end{align}.
$$

We desire to run simulations using these models for a variety of initial conditions.  However, this will eventually be computationally intensive as our models account for more factors and become more complex, so we have this second class, EBT, designed to mitigate this issue.

For EBT, we generate a population mean and a population covariance for the initial values for thermal temperature, leaf area index, and biomass, which EBT then uses to solve the differential equations to model the plant growth.  The process of selecting the population information, adapting the model class to fit with the EBT class, and running the class are discussed throughout.

Overall, second-order EBT does quite well, matching very closely with simulation using several different samples of initial value times.  This process made apparent a few changes we felt would be wise to keep in mind as we continue to modify our model of the wheat plant, especially related to growth rate of leaf area index and biomass.



In [1]:
%matplotlib inline
import pandas as pd                    # Import panda for reading env data
import numpy as np                     # Import array funtions
import math                            # Import all math functions
import matplotlib.pyplot as plt        # Import plotting functions
import ephem as em                     # Import pyephem to get daylight data
from datetime import *   
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import pchip
from scipy.integrate import odeint, RK45,ode
import scipy.linalg as sla
from autograd import grad, jacobian
from scipy.cluster.vq import kmeans2
!pip install sobol-seq
import sobol_seq

## Set up Model Class

In [2]:
"""Model Class Definition"""

class Model(object):
    
    #Attributes we want each instance of model to have
    def __init__(self,Eqns):
        self.Eqns = Eqns  #
        self.n_var = len(Eqns)
        self.solution=[]  # creates a new empty list for each model
        self.params=None   # Initialize parameter values
        
        self.y0=None      # Initialze initial state variables
        
        self.t=None       # Initialie array of times at which solutions are sought
        
        self.Env=None     # Set flag that the simulation environment is not yet defined
                          #    When it is, it will most likely be an array with one
                          #    row per measurement time and one column per environmental
                          #    variable.
        self.day_length=None
        
                    
    # Store a vector of desired solution times
    def new_soltimes(self,t):
        self.t=np.array(t)
    
    # Store a vector of desired state variable initial conditions
    def new_ICs(self,y0):
        self.y0=np.array(y0)
    
    # Store a vector of model parameters - this is used during parameter estimation
    def new_params(self,params):
        self.params=np.array(params)
        
    
    """ The following two methods are overloaded in the users class definition """
    
    # Fetch the environmental information from a file.  "Environ" is 
    # whatever information is needed to do this.
    def FetchEnviron(self,Environ):
        pass
    
    # Get the specific environment data for time t.  Depending on the time step
    # this might entail interpolation between observation times  The actual vector 
    # size will depend on model specifics.
    def GetEnvData(self,t):
        self.Edat=np.zeros(3)
        
        
    # Run the model
    def Run(self,ICs=None,params=None,t=None,Environ=None):
        
        # Perform any needed initializations
        if ICs is not None    : self.new_ICs(ICs)
        if params is not None : self.new_params(params)
        if t is not None      : self.new_soltimes(t)
        if Environ is not None: self.FetchEnviron(Environ)
        
        # Check if everything needed for a run is now present
        if self.params is None or self.y0 is None or self.t is None or self.Env is None:
            raise Exception("Prequisite data for run is incomplete")
        
        # Define the function that is passed to odeint
        def deriv(y,t,model):
            
            # Obtain the environmental data for this value of t
            self.Edat
            
            # Compute required derivatives.  This will entail using data that
            # should be ready and waiting in model.par and model.Edat
            return model.Eqns(y,t,model)

        # Call odeint to itegrate diffeq system.  All the odeint arguments are shown
        # here in case we want to modify something in a future version.  (It is, perhaps,
        # noteworthy that some of the extra parameters apparenly specify the order of 
        # integration to be peformed.)

        return odeint(deriv,           # Computes the derivative of y at t
                      self.y0,         # Initial condition on y (can be a vector) 
                      self.t,          # The time points for which to solve for y
                      args=(self,),    # Extra arguments to pass to function (self reference to
                                       #   this model instance)
                      Dfun=None,       # Gradient (Jacobian) of func 
                      col_deriv=0,     # True if Dfun defines derivatives down columns
                      full_output=0,   # True if to return a dictionary of 
                                       #   optional outputs as the second output
                      printmessg=0,    # Whether to print the convergence message
                      tfirst=False,    # If True, the first two arguments of func 
                                       #   (and Dfun, if given) must t, y
                                
        #  See https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
        #     ror the definition of these arguments
        
                      ml=None,           
                      mu=None, 
                      rtol=None, 
                      atol=None, 
                      tcrit=None, 
                      h0=0.0, 
                      hmax=0.0, 
                      hmin=0.0, 
                      ixpr=0, 
                      mxstep=0, 
                      mxhnil=0, 
                      mxordn=12, 
                      mxords=5)

## Defining Simple Wheat Subclass

This subclass takes a list of functions/equations as input, and then will use environmental input to keep track of thermal time and other parameters for plant growth.  It will also define rhs which will return a list of functions to be evaluated later.

In [3]:
class mark_0_wheat(Model):
    def __init__(self,Eqns):
        Model.__init__(self,Eqns)
        #self.n_var = Model.n_var
    def FetchEnviron(self, Environ):
        self.start_date = Environ[0]
        self.end_date = Environ[1]
        file = Environ[2]
    
        # reading the data
        w = pd.read_csv(file)

        # change this data into a numpy array to use the where function
        day_data = np.asarray(w)
    
        #find row index of selected start and stop dates, here we choose April 1st through November 11th
        start_index = int(np.where(day_data == self.start_date)[0])
        stop_index = int(np.where(day_data == self.end_date)[0])

        
        self.Env = day_data[start_index:stop_index+1, 2:].astype(np.float)
        
        self.t = np.arange(0,len(self.Env))          
        
        self.loc_data = self.Env
      
        self.T_avg  = pchip(self.t,(self.loc_data[:,0] + self.loc_data[:,1])/2)  #average temperature per day (max+min)/2
                                                                                 #use the pchip command to interpolate the
        self.I = pchip(self.t,(self.loc_data[:,-3])*0.0864)                      #the data since odeint takes noninteger vals
        
        self.Edat = [self.T_avg, self.I]
        
        def rhs(self, X, t):
            dTT = dtt(X, t) 
            dLAI = dlai(X, t)
            dB = dcho(X, t)       
            return np.array([dTT, dLAI, dB])


### Parameters for simple wheat functions

In [4]:
T_base =  0
alpha =   5 * 1.0125e-4   #correct alpha values

LAImax =  7.0
TTL =       21600/24
RUE =     1.5
#K =       0.7
K = 0.2        #correct value

TTM =    43200/24
#transition_period = 2 #time needed for lai and cho to transition to no growth
#lai_transition_tt = 20*transition_period  #thermal time needed for lai transition to no growth
#cho_transition_tt = 28*transition_period  #thermal time needed for cho transition to no growth
transition_period = 2
lai_transition_tt = 20*transition_period
cho_transition_tt = 28*transition_period


## Definition of Simple Wheat Functions 

Summary of changes:

We made two major adaptations to these functions and how they work with the model.  The first being that they are now defined outside of RHS and then are passed to RHS; this makes the model class a little more flexible in how it handles functions and smoothed over some type and name space issues we ran into when we first tried to merge our model class with EBT.

The second change we made was to simulate the piecewise aspect of dlai and dcho by using tanh as a sort of scaling factor in order to make it differentiable.  Specifically, we now define dlai as 
$$\dot{LAI} = \alpha T_{avg}LAI\cdot(LAI_{max}-LAI)*\frac{-1}{2}\tanh((TT - TTL)/TD_{lai})-1)$$
and dcho as 
$$ \dot{CHO}(t) = RUE[1-e^{-K\cdot LAI}]I*\frac{-1}{2}\tanh((TT - TTM)/TD_{cho})-1),$$
where $TD_{lai}$ and $TD_{cho}$ are thermal days required for the plant to  make the transition to no growth in leaf area index or biomass.  The whole plot was scaled by $\frac{-1}{2}$ since tanh has a width of two units and is flipped the opposite way from what we needed and shifted by the values $TTL$ and $TTM$ to produce the correct cutoffs.

The values needed to scale the inside of tanh were initially key in simply getting a good approximation of the original piecewise functions.  The larger the scale factor, the steeper the transition in tanh, and the more it looks like a piecewise function.  We promptly ran into the issue of a "RuntimeWarning" (with the warning citing overflow in power return, double_scalars, sinh, and cosh) for both of these when they were passed to odeint. We believe this was due to the transition being too sharp for autograd to differentiate it. To mitigate this, we began playing around with scaling by progressively smaller numbers in an effort to flatten out the transition to make it possible for autograd to work  with it. By trial and error, it was determined that the scale factor for dlai could be no larger than $\frac{1}{10}$ and the scale factor for dcho could be no larger than $\frac{1}{18}$.

Eventually, thanks to Dr. Nathan Albin, we realized that these scale factors should make biological sense and reflect the amount of time for the plant to cease growth in these characteristics.  Dr. Stephen Welch suggested going for 2 or 4-5 days.  Since the inside of tanh is in thermal time, these days needed to be converted to "thermal days".  To do this, we pinpointed the specific days that the transitions appeared to be taking place, day 57 for dlai and day 90 for dcho, and the rate of change of thermal time on each of those days was approximated using the graph of thermal time below (after running EBT).  Those slopes, 20 for lai and 28 for cho, were then used to compute the thermal transition time for dlai and dcho as shown in the cell above.



In [5]:
#Rate of change in Thermal Time
def dtt(X, t):
    return Sweet.T_avg(t)

#Rate of change in Leaf Area Index
def dlai(X, t):
    TT, LAI, CHO = X
    return alpha*Sweet.T_avg(t)*LAI*(LAImax - LAI)*(-1/2)*(np.tanh(((TT-TTL))/lai_transition_tt)-1) 
  
#Rate of change of carbohydrates
def dcho(X, t):
    TT, LAI, CHO = X
    return RUE*(1-np.exp(-K*LAI))*Sweet.I(t)*(-1/2)*(np.tanh((TT-TTM)/cho_transition_tt) - 1)  



## EBT model class

EBT model class takes in the desired model (an instance of Simple Wheat in our case), computes the gradient and hessian for the functions in the model, and then uses population statistics to determine a sort of average solution for the differential equations across possible initial values.

In [6]:
class EBTModel:
    '''
    Simple implementation of an EBT model.
    '''
    
    def __init__(self, model):
        '''
        Initializes a new EBTModel from a given ODE model.
        '''
        
        self.model = model
        self.n_var = model.n_var
        self.grad_f = [grad(f) for f in model.Eqns]
        self.hess_f = [jacobian(g) for g in self.grad_f]
        
    def rhs(self, t, X):
        '''
        The right-hand side function for the SECOND-ORDER EBT approximation.
        
        The order (t,X) is chosen to support the use of the new scipy.integrate
        ODE steppers.
        '''
        
        # initialize derivative vector
        deriv = np.zeros(X.shape)

        # unpack mu and (flattened) sigma
        mu     = X[:self.n_var]
        fsigma = X[self.n_var:]
        
        # unflatten sigma
        sigma  = fsigma.reshape(self.n_var,self.n_var)

        # evaluate derivatives of means
        for i in range(self.n_var):
            H = self.hess_f[i](mu, t).flatten()
            deriv[i] = self.model.Eqns[i](mu, t) + 0.5*H.dot(fsigma)

        # evaluate derivatives of covariance
        G = np.array([self.grad_f[i](mu, t) for i in range(self.n_var)])
        d_sigma = G.dot(sigma)
        d_sigma += d_sigma.T
        
        # flatten covariance derivatives
        deriv[self.n_var:] = d_sigma.flatten()

        return deriv

# Running EBT with Simple Wheat

We now create an instance of our simple wheat model, which we will call Sweet.  This instance of the model takes in the list of functions \[dtt, dlai, dcho\] we defined earlier.  Recall all parameters are defined several cells above.

In [7]:
'''Create instance of simple wheat model'''

Sweet=mark_0_wheat([dtt, dlai, dcho]) 


# Specify the environment to use.  
OurEnv = ('2018-04-01','2018-10-01','weather18.csv') 
Sweet.FetchEnviron(OurEnv)
                        
t= Sweet.t

# Creating instance of Ebt model using the simple wheat model 
ebt_sweet = EBTModel(Sweet)

FileNotFoundError: ignored

### Creating Initial Values
We produce initial values for EBT to use by randomly sampling "reasonable" values for mean and covariance for each of Thermal Time, Leaf Area Index, and Carbohydrates.  Then, using those means and covariances, we generate a distribution to then sample 100 potential values, and then use the means and covariances from those samples as our initial value data for EBT later.

The initial samplings of the mean are scaled by 0.05 to produce a tighter sampling and then we take the absolute value to guarantee positive values.  A similar strategy was adopted for the covariance, only this time using a scale factor of 0.01.  
There is some concern that these scaling factors are resulting in not enough increase in the values of leaf area index in the plots below, though this may also be due to the simplicity of the simple wheat model.  This issue will be discussed in more detail following the direct simulations of the samples for comparison.

In [ ]:
# seed the random number generator
seed = 0
np.random.seed(seed)

# number of samples to collect (this is N)
n_samples = 100

# random mean values for "populations" of initial thermal time, leaf area index, and carbohydrates; 
mu_0 = abs(0.05*np.random.randn(3))
mu_0=np.array([0.2,0.02,0.02])


# random covariance values for "populations" of initial thermal time, leaf area index, and carbohydrates
Q, _ = np.linalg.qr(np.random.randn(3,3))
sigma_0 = 0.01*Q.dot(np.diag(1 + 10*np.random.rand(3))).dot(Q.T)

#sigma_0=np.array([[0.0001, 0.00001,0.00],
#             [0.00001,0.0001 ,0.00],
#             [0.000  ,0.000  ,1.00]])

#sigma_0 = np.array([[0.00, 0.0001, 0.00001],
#                    [0.00,0.00001, 0.0001 ],
#                    [1.0, 0.000, 0.000]])
#

sigma_0 = np.array([
                    [0.07237154,0,0],[0,0.0001, 0.00001],[0, 0.00001, 0.0001]
                    ])
# sample the distribution to use to calculate an average and covariance to use for initial values
samples = abs(np.random.multivariate_normal(mu_0, sigma_0, n_samples))

# gather statistics for initial values
mu = np.average(samples, axis=0)
sigma = np.cov(samples, rowvar=False)

print('target mean = ')
print(mu_0)
print()
print('actual mean =')
print(mu)
print()
print('target covariance =')
print(sigma_0)
print()
print('actual covariance =')
print(sigma)
print()

# replace target values by true values for the test
mu_0 = mu
sigma_0 = sigma 

# Run odeint using second-order EBT

In [ ]:
# define length of time and number of intervals
t = np.linspace(0,175, 176)
n_t = len(t)

# create initial data using means and covariances computed in the cell abov
X0 = np.concatenate((mu_0, sigma_0.flatten()))

# a trick to swap (t,x) for (x,t) in order to use odeint; also sets up equations for EBT
rhs = lambda X,t : ebt_sweet.rhs(t, X)

# solve the ODE using second-order EBT with initial values and covariance computed in the cell above, X0
x = odeint(rhs, X0, t)

## Plots for time $(t)$ vs each of TT, LAI, and CHO

In [ ]:

dcho_xticks = [i*5 for i in range(int(175/5))]
    
plt.figure(figsize=(16,12))
plt.subplot(221)  
plt.ylim((0, 5000))
plt.plot(t, x[:,0])
plt.xlabel('time (day, beginning April 1)')
plt.ylabel('Thermal Time')
plt.grid(True)
#plt.yticks([i*100 for i in range(int(5000/100))])
#plt.xticks([i*5 for i in range(int(175/5))])

plt.subplot(222)  
plt.ylim((0, 1))
plt.plot(t, x[:,1])
plt.xlabel('time (day, beginning April 1)')
plt.ylabel('Leaf Area Index')
plt.grid(True)
#plt.xticks([i*5 for i in range(int(175/5))])

plt.subplot(223)  
plt.ylim((0, 80))
plt.plot(t, x[:,2])
plt.xlabel('time (day, beginning April 1)')
plt.ylabel('Biomass')
plt.grid(True)
#plt.yticks(range(31))
#plt.xticks(dcho_xticks)



These plots are exact what we expected for thermal time, leaf area index, and biomass.  Thermal time continually increases, the leaf area index stops increasing when the plant matures, as does the biomass. When compared with the plots that contain the piecewise versions of these functions (see homework 5), we see that the use of tanh wound up being a good approximation of the piecewise functions, even with the significant amount of flattening we had to do to avoid an overflow error with odein. 

### Direct simulation of the samples
We now evaluate the model at all the initial values we wanted to consider, instead of using a mean of the samples with their covariance to run EBT.

The first graph displays LAI for different intitial values and the second shows biomass relative to thermal time for different initial values.  We found it somewhat concerning, biologically, that the lines corresponding to different initial values never cross each other in either graph; this does not seem to reflect reality in that it could be possible for a plant to start with a smaller leaf area index and still wind up being larger than one that started with a larger leaf area index.  We also felt that the leaf area index should have a greater total increase, with the initial increase being quite steep and as time goes on, this increase tapering off, perhaps something similar to a square root or logarithmic function, which could be something to explore for the future.  We had similar thoughts about the biomass plots.

As expected, the thermal time plot isn't terribly revealing, since it is drawn from predetermined weather data in the file weather18.csv.



In [ ]:
# time range to solve on
t_final = 175
n_t = 176
t = np.linspace(0, t_final, n_t)

# storage place for solutions
# WARNING: this is just for testing, it wastes a LOT of space
X = np.zeros((n_samples, n_t, Sweet.n_var))

#swap input values for functions
rhs_no_ebt = lambda X,t : np.array([f(X,t) for f in Sweet.Eqns])


# find trajectory of each individual
from time import perf_counter
a = perf_counter()
for i in range(n_samples):
    X[i,:,:] = odeint(rhs_no_ebt, samples[i,:], t)
b = perf_counter()
print("time for direct", b-a)

    # draw the trajectories
plt.figure(figsize=(9,6))
plt.plot(X[:,:,0].T, X[:,:,0].T, color='#cccccc', linewidth=0.2);
plt.plot(X[:,0,0], X[:,0,0], '.', color='#00aa00', label='initial')
plt.plot(X[:,n_t//4,0], X[:,n_t//4,0], '.', color='#0000ff', label='t={:.3f}'.format(t[n_t//4]))
plt.plot(X[:,n_t//3,0], X[:,n_t//3,0], '.', color='#ffff00', label='t={:.3f}'.format(t[n_t//3]))
plt.plot(X[:,-1,0], X[:,-1,0], '.', color='#ff0000', label='t={:.3f}'.format(t[-1]))
plt.legend();

    
# draw the trajectories
plt.figure(figsize=(9,6))
plt.plot(X[:,:,0].T, X[:,:,1].T, color='#cccccc', linewidth=0.2);
plt.plot(X[:,0,0], X[:,0,1], '.', color='#00aa00', label='initial')
plt.plot(X[:,n_t//4,0], X[:,n_t//4,1], '.', color='#0000ff', label='t={:.3f}'.format(t[n_t//4]))
plt.plot(X[:,n_t//3,0], X[:,n_t//3,1], '.', color='#ffff00', label='t={:.3f}'.format(t[n_t//3]))
plt.plot(X[:,-1,0], X[:,-1,1], '.', color='#ff0000', label='t={:.3f}'.format(t[-1]))
plt.legend();

# draw the trajectories
plt.figure(figsize=(9,6))
plt.plot(X[:,:,0].T, X[:,:,2].T, color='#cccccc', linewidth=0.2);
plt.plot(X[:,0,0], X[:,0,2], '.', color='#00aa00', label='initial')
plt.plot(X[:,n_t//4,0], X[:,n_t//4,2], '.', color='#0000ff', label='t={:.3f}'.format(t[n_t//4]))
plt.plot(X[:,n_t//3,0], X[:,n_t//3,2], '.', color='#ffff00', label='t={:.3f}'.format(t[n_t//3]))
plt.plot(X[:,-1,0], X[:,-1,2], '.', color='#ff0000', label='t={:.3f}'.format(t[-1]))

plt.legend();

These plots show the progression of thermal time, LAI, and biomass with respect to time in days instead of thermal time.  

In [ ]:
#plt.ylim((0,1.0))

plt.plot(X[:,:,0].T)
plt.show()
plt.plot(X[:,:,1].T)
plt.show()
plt.plot(X[:,:,2].T)
plt.show()


# Comparison of first- and second-order EBT with Direct Simulations

The first-order EBT approximation is simply $\dot{\mu}=f(\mu,t)$.  It is easy to see, as expected, that first-order EBT does significantly worse than second-order EBT.  Second-order EBT turned out to be stunningly close to the average over the direct simulations, so close in fact we were initially concerned we had somehow duplicated the data. For this reason, the final plot displays the error between second-order EBT and the average over the direct simulations.  As would be expected, the error increases as time goes on, since it builds upon itself.  For LAI and biomass, the error flattens out when those values do as well, again, as expected.

It is interesting to note that the error for TT vascillates between positive and negative for a bit, and then  ultimately winds up positive, indicating that EBT tended to wind up overshooting the values, and that for LAI and biomass we see the opposite happening, with consistent underestimation occurring.



In [ ]:

X_avg = np.average(X, axis=0)   #average of direct simulations
#X_avg = np.var(X, axis=0)
#plot for thermal time
plt.figure(figsize=(9,6))
plt.plot(t, X_avg[:,0], 'k*', label='direct sim')
plt.plot(t, x[:,0], label='2nd order EBT')
plt.plot(t, X[0,:,0], label='1st order EBT')
plt.legend()
plt.title('Thermal Time')
plt.xlabel('t')
plt.ylabel('T')

#plot for leaf area index
plt.figure(figsize=(9,6))
plt.plot(t, X_avg[:,1], 'k*', label='direct sim')
plt.plot(t, x[:,1], label='2nd order EBT')
plt.plot(t, X[0,:,1], label='1st order EBT')
plt.legend()
plt.title('Leaf Area Index')
plt.xlabel('t')
plt.ylabel('LAI');

#plot for biomass
plt.figure(figsize=(9,6))
plt.plot(t, X_avg[:,2], 'k*', label='direct sim')
plt.plot(t, x[:,2], label='2nd order EBT')
plt.plot(t, X[0,:,2], label='1st order EBT')
plt.legend()
plt.title('Biomass')
plt.xlabel('t')
plt.ylabel('Biomass');

plt.figure(figsize = (9,6))
plt.plot(t, x[:,0] - X_avg[:,0], 'bo', label = 'error?')
plt.legend()
plt.title('difference in direct simulation and 2nd order EBT for TT')
plt.xlabel('t')
plt.ylabel('T T')

plt.figure(figsize = (9,6))
plt.plot(t, x[:,1] - X_avg[:,1], 'bo', label = 'error?')
plt.legend()
plt.title('difference in direct simulation and 2nd order EBT for LAI')
plt.xlabel('t')
plt.ylabel('T T')

plt.figure(figsize = (9,6))
plt.plot(t, x[:,2] - X_avg[:,2], 'bo', label = 'error?')
plt.legend()
plt.title('difference in direct simulation and 2nd order EBT for Biomass')
plt.xlabel('t')
plt.ylabel('T T')




LAI vs Biomass plot for 2nd Order EBT Simulation

In [ ]:
#plot for leaf on horizontal  vs biomass on vertical 
T = np.arange(176)
plt.figure(figsize=(9,6))
#plt.plot(X_avg[:,1], X_avg[:,2], 'r', label='direct sim')
#plt.plot(X[:,:,1].T, X[:,:,2].T, )
plt.plot(x[:,1], x[:,2],'k*', label='2nd order EBT')
plt.legend()
plt.title('Leaf area index vs Biomass')
plt.xlabel('leaf area index')
plt.ylabel('Biomass');


In [ ]:
X_var = np.var(X, axis=0)
T = np.arange(176)
plt.figure(figsize=(9,6))
plt.plot(X_var[:,1], X_var[:,2], 'r', label='variance')
plt.plot(X[:,:,1].T, X[:,:,2].T, )
plt.plot(x[:,1], x[:,2],'k*', label='2nd order EBT')
plt.plot(x[:,7], x[:,11], 'k*',label='test')
plt.legend()
plt.title('Leaf area index vs Biomass')
plt.xlabel('leaf area index')
plt.ylabel('Biomass');


In [ ]:
#plot for leaf on horizontal  vs biomass on vertical 

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
T = np.arange(176)
def plot_colourline(x,y,c, w=1):
    c = cm.jet((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i], lw = w)
    return c

def plot_colourline2(x,y,c, w=1):
    c = cm.jet((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i], lw = w)
    return c
plt.figure(figsize=(9,6))
plot_colourline(X[:,:,1].T, X[:,:,2].T, T)
plot_colourline2(x[:,1], x[:,2],T, w=5)


#plt.plot(X[:,:,1].T, X[:,:,2].T, label = "Direct simulation")
#plt.plot(x[:,1], x[:,2],'k*',c =c, label='2nd order EBT')
plt.legend()
plt.title('Leaf area index vs Biomass')
plt.xlabel('leaf area index')
plt.ylabel('Biomass');
plt.show()


In [ ]:

def plot_colourline3(x,y,c, w=1):
    #c = cm.jet((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=cm.jet(normalize(i*1.5)), lw = w)
    return c

def plot_colourline4(x,y,c, w=10):
    c = cm.cool((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=cm.cool(normalize(i*1.5)), lw = w)
    return c
def plot_colourline4dash(x,y,c, w=1):
    c = cm.cool((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=cm.Greys_r(normalize(i*1.5)), marker='o', lw = w)
    return c


#VARIANCE PLOTS FUNCTIONS     
def plot_colourline5(x,y,c, w=1):
    c = cm.ocean((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=cm.ocean(normalize(i*1.5)), lw = w)
    return c   
def plot_colourline6(x,y,c, w=1):
    c = cm.gist_earth((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=cm.Reds(normalize(i*1.5)), marker=11, lw = w)
    return c



In [ ]:
def coloured_linePlotter(x,y,c, w=1, marker=None, colorofline = cm.Reds, markersize=None):
    c = cm.gist_earth((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=colorofline(normalize(i*1.5)), marker=marker, lw = w, markersize=markersize)
    return c

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_avg = np.average(X, axis=0) 
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))
plot_colourline3(X[:,:,1].T, X[:,:,2].T, T)
plot_colourline4(x[:,1], x[:,2],T)
plot_colourline4dash(X_avg[:,1].T, X_avg[:,2].T, T)
plot_colourline5(X_var[:, 1], X_var[:, 2], T, w = 10)
plot_colourline6(x[:, 7], x[:, 11], T)

# coloured_linePlotter(X[:,:,1].T, X[:,:,2].T, T, w=1, colorofline = cm.jet, )
# coloured_linePlotter(x[:,1], x[:,2],T, w=10, colorofline=cm.cool)
# coloured_linePlotter(X_avg[:,1].T, X_avg[:,2].T, T, w=1, colorofline = cm.Greys_r, marker = 'o')
# coloured_linePlotter(X_var[:, 1], X_var[:, 2], T, w = 10, colorofline= cm.ocean)
# coloured_linePlotter(x[:, 7], x[:, 11], T, w=1, colorofline = cm.Reds, marker = 11)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.savefig("final.png", dpi = 300)
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 12})
import matplotlib.pyplot as plt
import matplotlib as mpl

fig, ax = plt.subplots(5,figsize=(6,6) )

norm = mpl.colors.Normalize(vmin=1, vmax=175)
cb1 = mpl.colorbar.ColorbarBase(ax[0], cmap=mpl.cm.jet,
                                norm=norm,
                                orientation='horizontal')
cb1.set_label('Time (days) - Population Direct Simulations')
cb2 = mpl.colorbar.ColorbarBase(ax[1], cmap=mpl.cm.cool,
                                norm=norm,
                                orientation='horizontal')

cb2.set_label('Time (days) -  Population Direct Simulation Mean')
cb3 = mpl.colorbar.ColorbarBase(ax[2], cmap=mpl.cm.Greys_r,
                                norm=norm,
                                orientation='horizontal')
cb3.set_label('Time (days) - 2nd Order EBT Mean')
cb4 = mpl.colorbar.ColorbarBase(ax[3], cmap=mpl.cm.ocean,
                                norm=norm,
                                orientation='horizontal')

cb4.set_label('Time(Days) - Population Direct Simulation Variance')
cb5 = mpl.colorbar.ColorbarBase(ax[4], cmap=mpl.cm.Reds,
                                norm=norm,
                                orientation='horizontal')

cb5.set_label('Time(days) - 2nd Order EBT variance')
plt.tight_layout()
fig.show()
plt.savefig('colorabars.png', dpi=300)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))

coloured_linePlotter(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.cool)


#varinces
coloured_linePlotter(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:, 7], x[:, 11], T, w=2, colorofline = cm.cool)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.savefig("final.png", dpi = 300)
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 12})
import matplotlib.pyplot as plt
import matplotlib as mpl

fig, ax = plt.subplots(4,figsize=(6,5) )

norm = mpl.colors.Normalize(vmin=1, vmax=175)


cb1 = mpl.colorbar.ColorbarBase(ax[0], cmap=mpl.cm.jet,
                                norm=norm,
                                orientation='horizontal')
cb1.set_label('Time (days) -  Population Direct Simulation Mean')

cb2 = mpl.colorbar.ColorbarBase(ax[1], cmap=mpl.cm.cool,
                                norm=norm,
                                orientation='horizontal')
cb2.set_label('Time (days) - 2nd Order EBT Mean')

cb3 = mpl.colorbar.ColorbarBase(ax[2], cmap=mpl.cm.jet,
                                norm=norm,
                                orientation='horizontal')
cb3.set_label('Time(Days) - Population Direct Simulation Variance')

cb4 = mpl.colorbar.ColorbarBase(ax[3], cmap=mpl.cm.cool,
                                norm=norm,
                                orientation='horizontal')

cb4.set_label('Time(days) - 2nd Order EBT variance')
plt.tight_layout()
fig.show()
plt.savefig('colorabars.png', dpi=300)


In [ ]:
k = x
p = []
for i in range(len(k)):
  p.append(k[i][3:12])
detty = []
for i in range(len(p)):
  detty.append(np.linalg.det(p[i].reshape((3,3))))
plt.figure(figsize=(9, 6))
plt.plot(detty)
plt.xlabel("time")
plt.ylabel("det")
plt.show()

In [ ]:
# x[0:175, 0:9][0]
a = x
p = []
detty = []
for i in range(len(a)):
  detty.append(np.linalg.det(a[:, 3:12][i].reshape((3,3))))
plt.figure(figsize=(9, 6))
plt.plot(detty)
plt.yscale('log')
plt.xlabel("time")
plt.ylabel("det")
plt.show()

In [ ]:
#saving numpy array
with open('original.npy', 'wb') as f:
    np.save(f, x)

In [ ]:
# x[0:175, 0:9][0]
a = np.load('original.npy')
# b = np.load('diverging.npy') 
b = x
p = []
q = []
detty = []
detty2 = []
for i in range(len(a)):
  detty.append(np.linalg.det(a[:, 3:12][i].reshape((3,3))))
for i in range(len(b)):
  detty2.append(np.linalg.det(b[:, 3:12][i].reshape((3,3))))
plt.figure(figsize=(9, 6))
plt.plot(detty, label = 'original parametres')
plt.plot(detty2, label = "diverging parametres")
plt.legend()
plt.yscale('log')
plt.xlabel("time")
plt.ylabel("det")
plt.savefig('detcompare.png', dpi=300)
plt.show()


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))

coloured_linePlotter(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.cool)


#varinces
coloured_linePlotter(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:, 7], x[:, 11], T, w=2, colorofline = cm.cool)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.twinx()
plt.ylabel('det')
plt.twiny()
plt.xlabel('time')
# plt.plot(detty2, label= "original paramenters", color = "k")
coloured_linePlotter(T, detty2, T, w=2, colorofline = cm.jet)
plt.legend()
plt.yscale('log')
#plt.plot(detty2, label= "diverging parametres paramenters", 'b')
plt.savefig("final.png", dpi = 300)
plt.show()

In [ ]:
# x[0:175, 0:9][0]
a = np.load('original.npy')
# b = np.load('diverging.npy') 
b = x
p = []
q = []
condy = []
condy2 = []
time = perf_counter()
for i in range(len(a)):
  condy.append(np.linalg.cond(a[:, 3:12][i].reshape((3,3))))
print("for condition number", (perf_counter()-time)/len(x))
for i in range(len(b)):
  condy2.append(np.linalg.cond(b[:, 3:12][i].reshape((3,3))))

plt.figure(figsize=(9, 6))
plt.plot(condy, label = 'original parametres')
plt.plot(condy2, label = "diverging parametres")
plt.legend()
plt.yscale('log')
plt.xlabel("time")
plt.ylabel("Condition Number")
plt.savefig('condcompare.png', dpi=300)
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))

coloured_linePlotter(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.jet)


#varinces
coloured_linePlotter(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:, 7], x[:, 11], T, w=2, colorofline = cm.jet)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.twinx()
plt.ylabel('Condition Number')
plt.twiny()
plt.xlabel('time')
# plt.plot(detty2, label= "original paramenters", color = "k")
coloured_linePlotter(T, condy2, T, w=2, colorofline = cm.jet)

plt.yscale('log')
#plt.plot(detty2, label= "diverging parametres paramenters", 'b')
plt.savefig("final.png", dpi = 300)
plt.show()

Creating own color bar

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))
def black_redcolourbar(x,y,c, w=1, marker=None, colorofline = cm.Reds, markersize=None):
    c = cm.Greys((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[175] if i!=40 else (1,0,0,1), marker=marker, lw = 3, markersize=markersize)
    return c

black_redcolourbar(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.jet)


#varinces
black_redcolourbar(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.jet, marker = '>', markersize=5)
black_redcolourbar(x[:, 7], x[:, 11], T, w=2, colorofline = cm.jet)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.axis([0, 2, 0, 100])
plt.twinx()
plt.ylabel('Condition Number')
plt.twiny()
plt.xlabel('time')
# plt.plot(detty2, label= "original paramenters", color = "k")
black_redcolourbar(T, condy2, T, w=2, colorofline = cm.jet)
plt.yscale('log')
#plt.plot(detty2, label= "diverging parametres paramenters", 'b')
plt.savefig("conditionNumber.png", dpi = 300)
plt.show()


In [ ]:
# x[0:175, 0:9][0]
a = np.load('original.npy')
# b = np.load('diverging.npy') 
b = x
p = []
q = []
Frobenius = []
Frobenius2  = []
time = perf_counter()
for i in range(len(a)):
  Frobenius.append(np.linalg.norm(a[:, 3:12][i].reshape((3,3)), ord='fro'))
print("for Frobenius number", (perf_counter()-time)/len(x))
for i in range(len(b)):
  Frobenius2.append(np.linalg.norm(b[:, 3:12][i].reshape((3,3)), ord='fro'))
plt.figure(figsize=(9, 6))
plt.plot(Frobenius , label = 'original parametres')
plt.plot(Frobenius2 , label = "diverging parametres")
plt.legend()
# plt.yscale('log')
plt.xlabel("time")
plt.ylabel("Frobenius norm")
plt.savefig('Frobeniusnorm.png', dpi=300)
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))

black_redcolourbar(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.prism, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.jet)


#varinces
black_redcolourbar(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.prism, marker = '>', markersize=5)
black_redcolourbar(x[:, 7], x[:, 11], T, w=2, colorofline = cm.jet)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.axis([0,2,0,100])
plt.twinx()
plt.ylabel('Frobenius Norm')
plt.twiny()
plt.xlabel('time')
black_redcolourbar(T, Frobenius2, T, w=2, colorofline = cm.jet)
# plt.yscale('log')
plt.savefig("Frobenius.png", dpi = 300)
plt.show()

In [ ]:
# x[0:175, 0:9][0]
a = np.load('original.npy')
# b = np.load('diverging.npy') 
b = x
p = []
q = []
secondnorm = []
secondnorm2  = []
time = perf_counter()
for i in range(len(a)):
  secondnorm.append(np.linalg.norm(a[:, 3:12][i].reshape((3,3)), ord=2))
print("for 2nd number", (perf_counter()-time)/len(a))
for i in range(len(b)):
  secondnorm2.append(np.linalg.norm(b[:, 3:12][i].reshape((3,3)), ord=2))
plt.figure(figsize=(9, 6))
plt.plot(secondnorm , label = 'original parametres')
plt.plot(secondnorm2 , label = "diverging parametres")
plt.legend()
# plt.yscale('log')
plt.xlabel("time")
plt.ylabel("second norm")
plt.savefig('secondnorm.png', dpi=300)
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
plt.rcParams.update({'font.size': 12})
normalize = matplotlib.colors.Normalize(vmin=1, vmax=175)
norm = matplotlib.colors.Normalize(vmin=1, vmax=175)
X_var = np.var(X, axis=0)
fig = plt.figure(figsize=(9, 6))

black_redcolourbar(X_avg[:,1].T, X_avg[:,2].T, T, w=2, colorofline = cm.jet, marker = '>', markersize=5)
coloured_linePlotter(x[:,1], x[:,2],T, w=2, colorofline=cm.jet)


#varinces
black_redcolourbar(X_var[:, 1], X_var[:, 2], T, w = 2, colorofline= cm.jet, marker = '>', markersize=5)
black_redcolourbar(x[:, 7], x[:, 11], T, w=2, colorofline = cm.jet)

import matplotlib.patches as mpatches
plt.title('Mean and variance plots of LAI vs Biomass')
plt.xlabel('Leaf Area Index')
plt.ylabel('Biomass');
plt.axis([0,2,0,100])
plt.twinx()
plt.ylabel('2- Norm')
plt.twiny()
plt.xlabel('time')
black_redcolourbar(T, secondnorm2, T, w=2, colorofline = cm.jet)
# plt.yscale('log')
plt.savefig("2-norm.png", dpi = 300)
plt.show()

In [ ]:
# sobol_seq.i4_bit_hi1
# sobol_seq.i4_bit_lo0
# sobol_seq.i4_sobol
# sobol_seq.i4_sobol_generate
# sobol_seq.i4_sobol_generate_std_normal
# sobol_seq.i4_uniform
# sobol_seq.prime_ge


In [ ]:
s2 = sobol_seq.i4_sobol_generate(2, 100)

sobol_X, sobol_Y = s2[:, 0], s2[:, 1]

sobol_X2 = (np.array(sobol_X) + np.random.uniform())%1
sobol_Y2 = (np.array(sobol_Y) + np.random.uniform())%1

SX = np.random.uniform(size=(100*2))

f, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(12,4))
ax1.scatter(SX[:100], SX[100:])
ax2.scatter(sobol_X, sobol_Y, color="red")
ax3.scatter(sobol_X2, sobol_Y2, color="green")

ax1.set_title("Random")
ax2.set_title("Sobol")
ax3.set_title("Sobol again")

In [ ]:
rs =[a.reshape(3,3) for a in x[:, 3:12]]

In [ ]:
vals= np.linalg.eigvals(rs)

In [ ]:
import sympy
a = sympy.Matrix(rs[0])
a.eigenvects()

In [ ]:
plt.figure(figsize =(9, 6))
plt.plot(vals[:, 0], label = '1st')
plt.plot(vals[:, 1], label = '2nd')
plt.plot(vals[:, 2] , label = '3rd')
plt.yscale('log')
plt.legend()
plt.xlabel('time')
plt.ylabel('eigenvalues')


In [ ]:
np.all(np.linalg.eigvals(rs) > 0)

In [ ]:
plt.plot(x[:, 3], label = 'TT')
plt.plot(x[:, 7], label ="LAI")
plt.plot(x[:, 11], label='biomass')
plt.legend()
plt.yscale("log")
plt.xlabel("time")
plt.ylabel("covaraince")

In [ ]:
plt.plot(x[:, 0], label = 'TT')
plt.plot(x[:, 1], label ="LAI")
plt.plot(x[:, 2], label='biomass')
plt.legend()
plt.yscale("log")
plt.xlabel("time")
plt.ylabel("mean")

In [ ]:
def series_of_points(vals):
  alpha = 0.1
  beta = 0.1
  gamma = 0.1
  Rx = []
  Ry = []
  Rz = []
  lams = []
  temp = []
  lamdot = []
  for t in range(len(vals)):
    Rx.append(np.array([[np.cos(alpha*t), -np.sin(alpha * t), 0], [np.sin(alpha*t), np.cos(alpha*t), 0], [0, 0, 1]]))
    Ry.append(np.array([[np.cos(beta*t), 0, np.sin(beta*t)], [0, 1, 0], [-np.sin(beta*t), 0, np.cos(beta*t)]]))
    Rz.append(np.array([[1, 0, 0], [0, np.cos(gamma*t), - np.sin(gamma*t)], [0, np.sin(gamma*t), np.cos(gamma*t)]]))
    lamdot.append(np.diag(vals[t]))
  return Rx, Ry, Rz, lamdot

In [ ]:
Rx, Ry, Rz, lamdot = series_of_points(vals)

In [ ]:
b = np.array([[1, 1, 1]])
Rdot = np.matmul(np.matmul(Rx, Ry), Rz)
A= []
for rdot, l in zip(Rdot, lamdot):
  A.append(np.matmul(np.matmul(rdot,l), rdot.T))

In [ ]:
points = []
for a in A:
  points.append(np.matmul(b, a)[0])
p = np.array(points)

In [ ]:
#%matplotlib notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
colo = []
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure(figsize=(9, 6))
G = gridspec.GridSpec(1, 2, width_ratios=[10, 1])
ax1 = fig.add_subplot(G[0:2,0], projection='3d')
for i in range(len(p) - 1):
  ax1.plot3D(p[i:i+2,0], p[i:i+2, 1], p[i:i+2, 2], c=cm.Reds(normalize(i)))
ax2 = fig.add_subplot(G[0,1])
cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.Reds,
                                 norm=norm,
                                 orientation='vertical')
cb1.set_label('time')
plt.tight_layout()
plt.show()

In [ ]:
from scipy.spatial.transform import Rotation as R
bunchofR = []
for i in range(len(vals)):
  r = R.from_quat([0, 0, np.sin(100*i), np.cos(100*i)])
  bunchofR.append(r.as_matrix())
A= []
for r, l in zip(bunchofR, lamdot):
  A.append(np.matmul(np.matmul(r,l), r.T))
points = []
for a in A:
  points.append(np.matmul(b, a)[0])
p2 = np.array(points)
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
colo = []
for k in range(len(vals)):
  colo.append(cm.Greens(normalize(k)))
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure(figsize=(9, 6))
G = gridspec.GridSpec(6, 2, width_ratios=[10, 1])
ax1 = fig.add_subplot(G[:,0], projection='3d')
ax1.scatter3D(p2[:,0], p2[:, 1], p2[:, 2])
#ax1.plot(p2[:,0], p2[:, 1], p2[:, 2])
ax2 = fig.add_subplot(G[1:6,1])

cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.Greens,
                                 norm=norm,
                                 orientation='vertical')
cb1.set_label('time')
plt.tight_layout()
plt.show()

In [ ]:
def angular_velocity(alpha, beta, gamma, vals):
  s = []
  for t in range(len(vals)):
    Rx = np.array([[np.cos(alpha*t), -np.sin(alpha * t), 0],
                   [np.sin(alpha*t), np.cos(alpha*t), 0], 
                   [0, 0, 1]])
    Ry = np.array([[np.cos(beta*t), 0, np.sin(beta*t)],
                   [0, 1, 0], 
                   [-np.sin(beta*t), 0, np.cos(beta*t)]])
    Rz = np.array([[1, 0, 0], 
                   [0, np.cos(gamma*t), - np.sin(gamma*t)], 
                   [0, np.sin(gamma*t), np.cos(gamma*t)]]) 
    Rxdot = np.array([[-np.sin(alpha *t)*alpha , -np.cos(alpha*t)*alpha , 0], 
                      [np.cos(alpha*t)*alpha, -np.sin(alpha*t)*alpha, 0], 
                      [0, 0, 0]])
    Rydot = np.array([[-np.sin(beta*t)*beta, 0, np.cos(beta*t)*beta], 
                      [0, 0, 0], 
                      [-np.cos(beta*t)*beta, 0, -np.sin(beta*t)*beta]])
    Rzdot = np.array([[0, 0, 0], 
                      [0, -np.sin(gamma*t)*gamma, - np.cos(gamma*t)*gamma], 
                      [0, np.cos(gamma*t)*gamma, np.sin(gamma*t)*gamma]])
    Rtx = Rx.T
    Rty = Ry.T
    Rtz = Rz.T 
    Rtxdot = Rxdot.T
    Rtydot = Rydot.T
    Rtzdot = Rzdot.T
    Rdot = np.matmul(np.matmul(Rxdot, Ry), Rz) + np.matmul(np.matmul(Rx, Rydot), Rz) + np.matmul(np.matmul(Rx, Ry), Rzdot)
    Rtdot = Rdot.T
    R = np.matmul(np.matmul(Rx, Ry), Rz)
    Rt = R.T
    l = [[3.63452908e-02*t, 0, 0], [0, 1.14151210e-04*t, 0], [0, 0, 6.86282637e-05*t]]
    ld = [[3.63452908e-02, 0, 0], [0, 1.14151210e-04, 0], [0, 0, 6.86282637e-05]]
    s.append(np.matmul(np.matmul(Rdot, l), Rt) + np.matmul(np.matmul(R, ld), Rt) + np.matmul(np.matmul(R, l), Rtdot))
  return s
S = angular_velocity(0.1, 0.1, 0.1, vals)

In [ ]:
def dA_dt(A, t):
  alpha, beta, gamma = 0.1, 0.1, 0.1 
  l = [[3.63452908e-02*t, 0, 0], [0, 1.14151210e-04*t, 0], [0, 0, 6.86282637e-05*t]]
  ld = [[3.63452908e-02, 0, 0], [0, 1.14151210e-04, 0], [0, 0, 6.86282637e-05]]
  Rx = np.array([[np.cos(alpha*t), -np.sin(alpha * t), 0],
                 [np.sin(alpha*t), np.cos(alpha*t), 0],
                 [0, 0, 1]])
  Ry = np.array([[np.cos(beta*t), 0, np.sin(beta*t)], 
                 [0, 1, 0], 
                 [-np.sin(beta*t), 0, np.cos(beta*t)]])
  Rz = np.array([[1, 0, 0], 
                 [0, np.cos(gamma*t), - np.sin(gamma*t)], 
                 [0, np.sin(gamma*t), np.cos(gamma*t)]]) 

  Rxdot = np.array([[-np.sin(alpha *t) *alpha, -np.cos(alpha*t) *alpha, 0],
                    [np.cos(alpha*t)*alpha, -np.sin(alpha*t)*alpha, 0], 
                    [0, 0, 0]])
  Rydot = np.array([[-np.sin(beta*t)*beta, 0, np.cos(beta*t)*beta],
                    [0, 0, 0],
                    [-np.cos(beta*t)*beta, 0, -np.sin(beta*t)*beta]])
  Rzdot = np.array([[0, 0, 0],
                    [0, -np.sin(gamma*t)*gamma, - np.cos(gamma*t)*gamma],
                    [0, np.cos(gamma*t)*gamma, np.sin(gamma*t)*gamma]])
  Rtx = Rx.T
  Rty = Ry.T
  Rtz = Rz.T 
  Rtxdot = Rxdot.T
  Rtydot = Rydot.T
  Rtzdot = Rzdot.T
  Rdot = np.matmul(np.matmul(Rxdot, Ry), Rz) + np.matmul(np.matmul(Rx, Rydot), Rz) + np.matmul(np.matmul(Rx, Ry), Rzdot)
  Rtdot = Rdot.T
  R = np.matmul(np.matmul(Rx, Ry), Rz)
  Rt = R.T
  S = np.matmul(np.matmul(Rdot, l), Rt) + np.matmul(np.matmul(R, ld), Rt) + np.matmul(np.matmul(R, l), Rtdot)
  return S.ravel()

In [ ]:
t = np.linspace(0, 176,175 )
Rx = np.array([[np.cos(0), -np.sin(0), 0], [np.sin(0), np.cos(0), 0], [0, 0, 1]])
Ry = np.array([[np.cos(0), 0, np.sin(0)], [0, 1, 0], [-np.sin(0), 0, np.cos(0)]])
Rz = np.array([[1, 0, 0], [0, np.cos(0), - np.sin(0)], [0, np.sin(0), np.cos(0)]])
# A0 = np.matmul(np.matmul(Rx, Ry), Rz).ravel() 
A0 = np.zeros(9)
As = odeint(dA_dt, A0, t)
As = np.array(As)

In [ ]:
su = [a.reshape(3,3) for a in As]
su = np.array(su)
sulli = []
for s in su:
  sulli.append(np.matmul(b, s)[0])
sulli = np.array(sulli) 

In [ ]:
#%matplotlib notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
colo = []
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure(figsize=(9, 6))
G = gridspec.GridSpec(1, 2, width_ratios=[10, 1])
ax1 = fig.add_subplot(G[0:2,0], projection='3d')
for i in range(len(sulli-1)):
  ax1.plot(sulli[i:i+2, 0], sulli[i:i+2, 1], sulli[i:i+2, 2], c = cm.jet(normalize(i)))
ax2 = fig.add_subplot(G[0,1])
cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.jet,
                                 norm=norm,
                                 orientation='vertical')
cb1.set_label('time')
plt.tight_layout()
plt.show()

In [ ]:
velocity = []
for s in S:
  velocity.append(np.matmul(b, s)[0])
vel = np.array(velocity)

In [ ]:
#%matplotlib notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
colo = []
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure(figsize=(9, 6))
G = gridspec.GridSpec(1, 2, width_ratios=[10, 1])
ax1 = fig.add_subplot(G[0:2,0], projection='3d')
for i in range(len(vel) - 1):
  ax1.plot3D(vel[i:i+2,0], vel[i:i+2, 1], vel[i:i+2, 2], c=cm.Reds(normalize(i)))

ax2 = fig.add_subplot(G[0,1])
cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.Reds,
                                 norm=norm,
                                 orientation='vertical')
cb1.set_label('time')
plt.tight_layout()
plt.show()

In [ ]:
# x, y, z = p[:, 0], p[:, 1], p[:, 2] 
x, y, z = sulli[:, 0], sulli[:, 1], sulli[:, 2] 
u, v, w = vel[:, 0], vel[:, 1], vel[:, 2]
mpl.rcParams['legend.fontsize'] = 10
#fig = plt.figure(figsize=(9, 6), dpi=1200)
fig = plt.figure(figsize=(9, 6))
G = gridspec.GridSpec(6, 2, width_ratios=[10, 1])
ax1 = fig.add_subplot(G[:,0], projection='3d')
for i in range(len(p) - 1):
  ax1.plot3D(sulli[i:i+2,0], sulli[i:i+2, 1], sulli[i:i+2, 2], c=cm.Reds(normalize(i)))
ax1.quiver(x[1::5], y[1::5], z[1::5], u[1::5], v[1::5], w[1::5],length=1, arrow_length_ratio=0.2, normalize=True)
ax2 = fig.add_subplot(G[1:6,1])
cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.Reds,
                                norm=norm,
                                orientation='vertical')
cb1.set_label('time')
plt.tight_layout()
# plt.savefig("8-5-2020", dpi = 1200)
# plt.show()

In [ ]:
# def dlamda_dA(lamda, A):
#   print(A)
#   print(lamda)
#   lamda, v = np.linalg.eigh(np.array(A).reshape(3, 3)) 
#   dlamda_dA = np.matmul(v, v.T)
#   return dlamda_dA.ravel()
# lamda0 = [3.63452908e-02, 1.14151210e-04, 6.86282637e-05]
# lams = odeint(dlamda_dA, lamda0, As)

In [ ]:
# def dA_dlamda(A, lamda, su):
#   l, v = np.linalg.eig(A.reshape(3,3))
#   a = np.linalg.inv(np.matmul(v.T, v))
#   return a.ravel()
# lamdas = np.linspace(0,50, 51)
# a0 = np.diag([1, 1,1]).ravel()
# AAs = odeint(dA_dlamda, a0, lamdas)

In [ ]:
# su = [a.reshape(3,3) for a in aas]
# su = np.array(su)
# sulli = []
# for s in su:
#   sulli.append(np.matmul(b, s)[0])
# sulli = np.array(sulli) 
# import matplotlib as mpl
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import gridspec
# colo = []
# mpl.rcParams['legend.fontsize'] = 10
# fig = plt.figure(figsize=(9, 6))
# G = gridspec.GridSpec(1, 2, width_ratios=[10, 1])
# ax1 = fig.add_subplot(G[0:2,0], projection='3d')
# for i in range(len(sulli-1)):
#   ax1.plot(sulli[i:i+2, 0], sulli[i:i+2, 1], sulli[i:i+2, 2], c = cm.Reds(normalize(i)))

# ax2 = fig.add_subplot(G[0,1])
# cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cm.Reds,
#                                  norm=norm,
#                                  orientation='vertical')
# cb1.set_label('time')
# plt.tight_layout()
# plt.show()